In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pyperclip

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
hbg = pd.read_csv("./data/hbg_result1591.csv", index_col=0)

In [3]:
h = hbg.copy()
h.head()

,Contents,Like,ID,Tags
0,"""연봉 100억"" 일타강사 이지영,,다이어트 방법데일리 업데잇 @book5657s ...",12,book5657s,"['#약수핫플', '#경성대맛집', '#양천구떡케이크', '#신촌네일', '#동안구..."
1,?세계적인 스타 셰프 고든램지의 '고든 램지 버거'가 캐주얼 콘셉트로 새롭게 '고든...,395,etc_zine,[]
2,.아마도 전국에서 젤 유명한 옛날스타일 햄버거라죠??송탄미군부대 앞에 있는 #미스진...,1517,songs_kitchens,"['#미스진버거미군들한테', '#송탄', '#미스진버거??스페셜A(7', '#송탄맛..."
3,?<통새우폭탄버거>?????????????????먹시인입 속을 가득 채우는 탱글한 ...,4265,mukseein,"['#왓더버거??', '#직화통새우버거', '#셧더버거', '#햄버거맛집', '#새..."
4,꿀 같은데 헬인 알바 모음??@@카페 알바 누가 꿀이래ㅜㅜ.오늘의 꿀팁은?? @la...,664,lady_good_tip,"['#헬', '#헬인알바', '#꿀알바', '#알바모음', '#아르바이트', '#카..."


In [4]:
h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1591 entries, 0 to 1590
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Contents  1591 non-null   object
 1   Like      1591 non-null   object
 2   ID        1591 non-null   object
 3   Tags      1591 non-null   object
dtypes: object(4)
memory usage: 62.1+ KB


In [5]:
# contents 특수기호 및 문자 제거
h["Contents"] = h["Contents"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

# # tags # 제외한 특수 기호 제거
h["Tags"] = h["Tags"].str.strip("]['")
h["Tags"] = h["Tags"].str.replace("'", repl=r"", regex=True)

In [6]:
h.head()

,Contents,Like,ID,Tags
0,연봉 100억 일타강사 이지영 다이어트 방법데일리 업데잇 book5657s ...,12,book5657s,"#약수핫플, #경성대맛집, #양천구떡케이크, #신촌네일, #동안구맛집추천, #수유데..."
1,세계적인 스타 셰프 고든램지의 고든 램지 버거 가 캐주얼 콘셉트로 새롭게 고든...,395,etc_zine,
2,아마도 전국에서 젤 유명한 옛날스타일 햄버거라죠 송탄미군부대 앞에 있는 미스진...,1517,songs_kitchens,"#미스진버거미군들한테, #송탄, #미스진버거??스페셜A(7, #송탄맛집, #옛날햄버..."
3,통새우폭탄버거 먹시인입 속을 가득 채우는 탱글한 ...,4265,mukseein,"#왓더버거??, #직화통새우버거, #셧더버거, #햄버거맛집, #새우버거, #버거맛집..."
4,꿀 같은데 헬인 알바 모음 카페 알바 누가 꿀이래ㅜㅜ 오늘의 꿀팁은 la...,664,lady_good_tip,"#헬, #헬인알바, #꿀알바, #알바모음, #아르바이트, #카페, #배스킨, #햄버..."


#### 2_1) like 전처리 : 평균

In [7]:
# 중간값
num = h[~(h["Like"].str.contains("좋아합니")|h["Like"].str.contains("눌러보세"))]["Like"]
num = num.astype(int)
num.sum()/len(num)    # 92.5

92.46656649135988

In [8]:
h["Like"].str.contains("좋아합니").value_counts()

False    1338
True      253
Name: Like, dtype: int64

In [9]:
h["Like"].str.contains("눌러보세").value_counts()

False    1584
True        7
Name: Like, dtype: int64

In [10]:
like_str_idx = h[h["Like"].str.contains("좋아합니")].index
like_0_idx = h[h["Like"].str.contains("눌러보세")].index

In [11]:
for i in range(len(h)):
    for j in like_str_idx:
        for k in like_0_idx:
            if i==j:
                h["Like"][j]=92.5
            elif i==k:
                h["Like"][k]=0
            else:
                continue

In [12]:
h["Like"].str.contains("좋아합니").value_counts()

False    1331
Name: Like, dtype: int64

In [13]:
h["Like"].str.contains("눌러보세").value_counts()

False    1331
Name: Like, dtype: int64

In [14]:
h["Like"] = h["Like"].astype("int")

In [15]:
h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1591 entries, 0 to 1590
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Contents  1591 non-null   object
 1   Like      1591 non-null   int32 
 2   ID        1591 non-null   object
 3   Tags      1591 non-null   object
dtypes: int32(1), object(3)
memory usage: 120.5+ KB


In [154]:
# h.to_csv("./data/hbg1_avg.csv")

#### 2_2) like 전처리 : 중간값 (h 다시 불러오기)

In [100]:
# 중간값
num = h[~(h["Like"].str.contains("좋아합니")|h["Like"].str.contains("눌러보세"))]["Like"]
num = num.astype(int)
num.median()

25.0

In [101]:
h["Like"].str.contains("좋아합니").value_counts()

False    1338
True      253
Name: Like, dtype: int64

In [102]:
h["Like"].str.contains("눌러보세").value_counts()

False    1584
True        7
Name: Like, dtype: int64

In [103]:
like_str_idx = h[h["Like"].str.contains("좋아합니")].index
like_0_idx = h[h["Like"].str.contains("눌러보세")].index

In [105]:
for i in range(len(h)):
    for j in like_str_idx:
        for k in like_0_idx:
            if i==j:
                h["Like"][j]=25
            if i==k:
                h["Like"][k]=0
            else:
                continue

In [106]:
h["Like"].str.contains("좋아합니").value_counts()

False    1331
Name: Like, dtype: int64

In [107]:
h["Like"].str.contains("눌러보세").value_counts()

False    1331
Name: Like, dtype: int64

In [108]:
h["Like"] = h["Like"].astype("int")

In [109]:
h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1591 entries, 0 to 1590
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Contents  1591 non-null   object
 1   Like      1591 non-null   int32 
 2   ID        1591 non-null   object
 3   Tags      1591 non-null   object
dtypes: int32(1), object(3)
memory usage: 120.5+ KB


In [111]:
# h.to_csv("./data/hbg1_med.csv")

In [16]:
# Like 정규화 및 평균 이하 제거 

h_like = h["Like"].copy()
h_like = pd.DataFrame(h_like)
h_like.head()

,Like
0,12
1,395
2,1517
3,4265
4,664


In [17]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxcaler객체 생성
scaler = MinMaxScaler()

# MinMaxScaler 로 데이터 셋 변환 .fit() 과 .transform() 호출.  
scaler.fit(h_like)
h_like_scaled = scaler.transform(h_like)

# transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
like_scaled = pd.DataFrame(data=h_like_scaled)

In [18]:
like_scaled.columns=["like_scaled"]
like_scaled.head()

,like_scaled
0,0.002076
1,0.068339
2,0.262457
3,0.737889
4,0.114879


In [19]:
h_scaled = pd.concat([h, like_scaled], axis=1)
h_scaled

,Contents,Like,ID,Tags,like_scaled
0,연봉 100억 일타강사 이지영 다이어트 방법데일리 업데잇 book5657s ...,12,book5657s,"#약수핫플, #경성대맛집, #양천구떡케이크, #신촌네일, #동안구맛집추천, #수유데...",0.002076
1,세계적인 스타 셰프 고든램지의 고든 램지 버거 가 캐주얼 콘셉트로 새롭게 고든...,395,etc_zine,,0.068339
2,아마도 전국에서 젤 유명한 옛날스타일 햄버거라죠 송탄미군부대 앞에 있는 미스진...,1517,songs_kitchens,"#미스진버거미군들한테, #송탄, #미스진버거??스페셜A(7, #송탄맛집, #옛날햄버...",0.262457
3,통새우폭탄버거 먹시인입 속을 가득 채우는 탱글한 ...,4265,mukseein,"#왓더버거??, #직화통새우버거, #셧더버거, #햄버거맛집, #새우버거, #버거맛집...",0.737889
4,꿀 같은데 헬인 알바 모음 카페 알바 누가 꿀이래ㅜㅜ 오늘의 꿀팁은 la...,664,lady_good_tip,"#헬, #헬인알바, #꿀알바, #알바모음, #아르바이트, #카페, #배스킨, #햄버...",0.114879
...,...,...,...,...,...
1586,,6,s_reetl_ght,,0.001038
1587,2023 1 7,292,itzel_day_,,0.050519
1588,제주 판타스틱버거 제주특별자치도 서귀포시 표선면 토산중앙로15번길 6 ...,180,gwimjjijji,"#판타스틱버거, #제주판타스틱버거, #제주수제버거, #수제버거, #햄버거, #버거맛...",0.031142
1589,최애햄버거 더블패티는 기본 fiveguysburgers 파이브가이즈 햄버거,15,yongseongjhkim,"#fiveguysburgers, #파이브가이즈, #햄버거",0.002595


In [20]:
# 보기 좋은 순서로 변경~
h_scaled = h_scaled[["ID", "Contents", "Tags", "Like", "like_scaled"]]
h_scaled

,ID,Contents,Tags,Like,like_scaled
0,book5657s,연봉 100억 일타강사 이지영 다이어트 방법데일리 업데잇 book5657s ...,"#약수핫플, #경성대맛집, #양천구떡케이크, #신촌네일, #동안구맛집추천, #수유데...",12,0.002076
1,etc_zine,세계적인 스타 셰프 고든램지의 고든 램지 버거 가 캐주얼 콘셉트로 새롭게 고든...,,395,0.068339
2,songs_kitchens,아마도 전국에서 젤 유명한 옛날스타일 햄버거라죠 송탄미군부대 앞에 있는 미스진...,"#미스진버거미군들한테, #송탄, #미스진버거??스페셜A(7, #송탄맛집, #옛날햄버...",1517,0.262457
3,mukseein,통새우폭탄버거 먹시인입 속을 가득 채우는 탱글한 ...,"#왓더버거??, #직화통새우버거, #셧더버거, #햄버거맛집, #새우버거, #버거맛집...",4265,0.737889
4,lady_good_tip,꿀 같은데 헬인 알바 모음 카페 알바 누가 꿀이래ㅜㅜ 오늘의 꿀팁은 la...,"#헬, #헬인알바, #꿀알바, #알바모음, #아르바이트, #카페, #배스킨, #햄버...",664,0.114879
...,...,...,...,...,...
1586,s_reetl_ght,,,6,0.001038
1587,itzel_day_,2023 1 7,,292,0.050519
1588,gwimjjijji,제주 판타스틱버거 제주특별자치도 서귀포시 표선면 토산중앙로15번길 6 ...,"#판타스틱버거, #제주판타스틱버거, #제주수제버거, #수제버거, #햄버거, #버거맛...",180,0.031142
1589,yongseongjhkim,최애햄버거 더블패티는 기본 fiveguysburgers 파이브가이즈 햄버거,"#fiveguysburgers, #파이브가이즈, #햄버거",15,0.002595


In [36]:
h_scaled["Tags"].str.split("#")[0]

['',
 '약수핫플, ',
 '경성대맛집, ',
 '양천구떡케이크, ',
 '신촌네일, ',
 '동안구맛집추천, ',
 '수유데이트코스, ',
 '자연산막회, ',
 '대학생코디, ',
 '장어, ',
 '성신여대맛집, ',
 '카페맛집, ',
 '스타벅스, ',
 '부암동네일, ',
 '청담눈썹문신, ',
 '햄버거, ',
 '노원핫플, ',
 '건대눈썹문신, ',
 '보라매눈썹문신, ',
 '강동구네일, ',
 '일산가로수길']

In [37]:
tag_sum=[]
for i in range(len(h_scaled)):
    if len(h_scaled["Tags"].str.split("#")[1]) == 1:
        tag_sum.append(len(h_scaled["Tags"].str.split("#")[i][1:]))
        
h_scaled["tag_sum"] = tag_sum

C:\Users\somin\AppData\Local\Temp\ipykernel_23564\1683845082.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_scaled["tag_sum"] = tag_sum


In [38]:
h_scaled.head()

,ID,Contents,Tags,Like,like_scaled,tag_sum
0,book5657s,연봉 100억 일타강사 이지영 다이어트 방법데일리 업데잇 book5657s ...,"#약수핫플, #경성대맛집, #양천구떡케이크, #신촌네일, #동안구맛집추천, #수유데...",12,0.002076,20
1,etc_zine,세계적인 스타 셰프 고든램지의 고든 램지 버거 가 캐주얼 콘셉트로 새롭게 고든...,,395,0.068339,0
2,songs_kitchens,아마도 전국에서 젤 유명한 옛날스타일 햄버거라죠 송탄미군부대 앞에 있는 미스진...,"#미스진버거미군들한테, #송탄, #미스진버거??스페셜A(7, #송탄맛집, #옛날햄버...",1517,0.262457,10
3,mukseein,통새우폭탄버거 먹시인입 속을 가득 채우는 탱글한 ...,"#왓더버거??, #직화통새우버거, #셧더버거, #햄버거맛집, #새우버거, #버거맛집...",4265,0.737889,9
4,lady_good_tip,꿀 같은데 헬인 알바 모음 카페 알바 누가 꿀이래ㅜㅜ 오늘의 꿀팁은 la...,"#헬, #헬인알바, #꿀알바, #알바모음, #아르바이트, #카페, #배스킨, #햄버...",664,0.114879,12


In [ ]:
# h_scaled_avg.to_csv("./data/test_hbg.csv")